In [ ]:
# Steps to install
# 1. pip install sqlalchemy-bigquery 
    #google-cloud-bigquery-storage 
    #pyarrow
# 2. Copy the credentials file to wherever you set BIGQUERY_CREDENTIALS_PATH to

In [ ]:
import json
import os
import pandas as pd
import pyarrow

import sys
from os import path
import numpy

from dotenv import load_dotenv
from sqlalchemy import create_engine


load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')

In [ ]:
sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
import create_queries
%load_ext autoreload
%autoreload 2

In [ ]:
import prepare_data

### Below edit for customer abbreviation and today's date

In [ ]:
COMPANY_NAME = 'DP'
COMPANY_IDENTIFIER = 'DP'


INVENTORY_DATE = '11/1/2021'

In [ ]:
company_inventory_packages_query = create_queries.create_company_inventory_packages_query(COMPANY_IDENTIFIER)

engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))
company_inventory_packages_dataframe = pd.read_sql_query(company_inventory_packages_query, engine)

In [ ]:
company_inventory_packages_dataframe['is_trade_sample'].value_counts()

## create code to upload inventory report to the inventory_month_end SQL table

In [ ]:
from sqlalchemy import create_engine
import urllib
import pyodbc
import datetime as dt

In [ ]:
# Bespoke Engine
params = urllib.parse.quote_plus("DRIVER={ODBC Driver 17 for SQL Server};"
                                 "SERVER=bespoke-database-1.cmevrozrcs7c.us-west-2.rds.amazonaws.com;"
                                 "DATABASE=metrc;"
                                 "UID=admin;"
                                 "PWD=N19lrqxnurTUJLJT6GFe")
engine = create_engine("mssql+pyodbc:///?odbc_connect={}".format(params))
engine_con = engine.connect()

In [ ]:
this_date = dt.date(2021, 11, 1)

this_year = this_date.year
this_month = this_date.strftime('%m')
this_day = this_date.strftime('%d')

In [ ]:
abbreviations_df = pd.read_sql('abbreviations', con=engine_con)

In [ ]:
for n in range(len(abbreviations_df)):
    
#     print(f"{abbreviations_df['abbreviation'].iloc[n]} is short for {abbreviations_df['company'].iloc[n]}")
    
    COMPANY_NAME = abbreviations_df['company'].iloc[n]
    COMPANY_IDENTIFIER = abbreviations_df['abbreviation'].iloc[n]
    company_name = COMPANY_NAME.lower().replace(' ', '_')
    
    company_inventory_packages_query = create_queries.create_company_inventory_packages_query(COMPANY_IDENTIFIER)
    
    engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))
    company_inventory_packages_dataframe = pd.read_sql_query(company_inventory_packages_query, engine)
    
    company_inventory_packages_dataframe['abbreviation'] = COMPANY_IDENTIFIER
    company_inventory_packages_dataframe['inventory_report_date'] = this_date
    
    company_inventory_packages_dataframe['inventory_report_date'] = pd.to_datetime(company_inventory_packages_dataframe['inventory_report_date'])
    company_inventory_packages_dataframe['packaged_date'] = pd.to_datetime(company_inventory_packages_dataframe['packaged_date'])
    company_inventory_packages_dataframe['archived_date'] = pd.to_datetime(company_inventory_packages_dataframe['archived_date'])
    company_inventory_packages_dataframe['finished_date'] = pd.to_datetime(company_inventory_packages_dataframe['finished_date'])
    
    company_inventory_packages_dataframe.to_sql('inventory_month_end', con=engine_con, if_exists='append', index=False)
    
    print(f'{COMPANY_NAME} is added')
    print(f'{COMPANY_NAME} was {len(company_inventory_packages_dataframe)} entries long')